# Data import

Common tomography data formats are imported through [DXchange](http://dxchange.readthedocs.org/en/latest/source/api/dxchange.exchange.html) (available through Conda):


In [ ]:
import dxchange as dx

In [ ]:
prj, flat, dark, ang = dx.read_aps_32id('data/data-simulated.h5')

# Tomographic image reconstruction

## Direct reconstruction algorithms

* Filtered-Backprojection with interpolation on Fourier space (Gridrec)
* Filtered-Backprojection with interpolation on real space (FBP)

In [ ]:
import tomopy
import matplotlib.pyplot as plt

SMALL_SIZE = 8
MEDIUM_SIZE = 8
BIGGER_SIZE = 10

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rc('font', family='Times New Roman')
plt.rcParams["mathtext.fontset"] = 'stix'

rec = dict()  # Store all of the different reconstructions here

In [ ]:
rec['gridrec'] = tomopy.recon(prj, ang, algorithm='gridrec')

In [ ]:
plt.figure(dpi=150)
plt.imshow(rec['gridrec'][0], cmap='gray', interpolation='none')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

One can define the shape of the final image reconstruction:

In [ ]:
rec['gridrec'] = tomopy.recon(prj, ang, algorithm='gridrec', num_gridx=128, num_gridy=128)

In [ ]:
plt.figure(dpi=150)
plt.imshow(rec['gridrec'][0], cmap='gray', interpolation='none')
plt.xlabel('Detector Pixels (translation)')
plt.ylabel('Rotation angles (rotation)')
plt.show()

### Choosing filters

Available filters:

* Ramp
* Shepp-Logan
* Butterworth
* Parzen
* Cosine
* Hamming

In [ ]:
import numpy as np

# low-pass Butterworth filter parameters
cutoff = 0.5  # [0, 1]
order = 2  # 0, 1, 2, 3, ...

x = np.linspace(-1, 1, 128)
y = 1 / (1 + np.power(x / cutoff, 2 * order))

plt.figure()
plt.plot(x, y)
plt.axis('tight')
plt.grid('on')
plt.show()

In [ ]:
rec['gridrec'] = tomopy.recon(
    prj, ang, algorithm='gridrec', 
    num_gridx=128, num_gridy=128, 
    filter_name='butterworth', filter_par=[0.5, 2.],
)

In [ ]:
plt.figure(dpi=150)
plt.imshow(rec['gridrec'][0], cmap='gray', interpolation='none')
plt.xlabel('Detector Pixels (translation)')
plt.ylabel('Rotation angles (rotation)')
plt.show()

## Iterative reconstrucrtion algorithms

### Without penalization

#### Kaczmarz methods:

* Algebrais Reconstruction Technique (ART)
* Simultaneous Iterative Reconstruction Technique (SIRT)
* Block Algebraic Reconstruction (BART)

In [ ]:
for algorithm in ['art', 'sirt', 'bart']:
    rec[algorithm] = tomopy.recon(
            prj, ang, algorithm=algorithm, 
            num_gridx=128, num_gridy=128,
            num_iter=10
    )

In [ ]:
plt.figure(dpi=150, tight_layout=True)
plt.subplot(131)
plt.imshow(rec['art'][0], cmap='gray', interpolation='none')
plt.title('ART')
plt.xlabel('x')
plt.ylabel('y')
plt.subplot(132)
plt.imshow(rec['sirt'][0], cmap='gray', interpolation='none')
plt.title('SIRT')
plt.xlabel('x')
plt.ylabel('y')
plt.subplot(133)
plt.imshow(rec['bart'][0], cmap='gray', interpolation='none')
plt.title('BART')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

#### Expectation maximization algorithms:

* Maximum-Likelihood Expectation-Maximization (MLEM)
* Ordered Subset Expectation-Maximization (OSEM)

In [ ]:
for algorithm in ['mlem', 'osem']:
    rec[algorithm] = tomopy.recon(
        prj, ang, algorithm=algorithm, 
        num_gridx=128, num_gridy=128,
        num_iter=10
    )

In [ ]:
plt.figure(dpi=150, tight_layout=True)
plt.subplot(131)
plt.imshow(rec['mlem'][0], cmap='gray', interpolation='none')
plt.title('MLEM')
plt.xlabel('x')
plt.ylabel('y')
plt.subplot(132)
plt.imshow(rec['osem'][0], cmap='gray', interpolation='none')
plt.title('OSEM')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

### With quadratic penalization

#### Expectation maximization algorithms:

* Penalized Maximum-Likelihood with Quadratic penalty (PML_QUAD)
* Ordered Subset Penalized Maximum-Likelihood with Quadratic penalty (OSMPL_QUAD)

In [ ]:
# PML_QUAD
rec4 = tomopy.recon(prj, ang, algorithm='pml_quad', 
                   num_gridx=128, num_gridy=128,
                   num_iter=20,
                   reg_par=0.1)

rec5 = tomopy.recon(prj, ang, algorithm='pml_quad', 
                   num_gridx=128, num_gridy=128,
                   num_iter=20,
                   reg_par=100)

In [ ]:
plt.figure(dpi=150, tight_layout=True)
plt.subplot(131)
plt.imshow(rec4[0], cmap='gray', interpolation='none')
plt.title('PML_QUAD, reg_par=0.1')
plt.xlabel('x')
plt.ylabel('y')
plt.subplot(132)
plt.imshow(rec5[0], cmap='gray', interpolation='none')
plt.title('PML_QUAD, reg_par=100')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

### With hybrid (mixed quadratic-linear) penalization

#### Expectation maximization algorithms:

* Penalized Maximum-Likelihood with Hybrid penalty (PML_HYBRID)
* Ordered Subset Penalized Maximum-Likelihood with Hybrid penalty (OSMPL_QUAD)

In [ ]:
# OSPML_HYBRID
rec4 = tomopy.recon(prj, ang, algorithm='ospml_hybrid', 
                   num_gridx=128, num_gridy=128,
                   num_iter=20,
                   reg_par=[1, 0.1])

rec5 = tomopy.recon(prj, ang, algorithm='ospml_hybrid', 
                   num_gridx=128, num_gridy=128,
                   num_iter=20,
                   reg_par=[0.1, 1],
                   num_block=10)

rec6 = tomopy.recon(prj, ang, algorithm='ospml_hybrid', 
                   num_gridx=128, num_gridy=128,
                   num_iter=20,
                   reg_par=[0.1, 0.1],
                   num_block=10)

In [ ]:
plt.figure(dpi=150, tight_layout=True)
plt.subplot(131)
plt.imshow(rec4[0], cmap='gray', interpolation='none')
plt.title('OSPML_HYBRID, reg_par=[1, 0.1]')
plt.xlabel('x')
plt.ylabel('y')
plt.subplot(132)
plt.imshow(rec5[0], cmap='gray', interpolation='none')
plt.title('OSPML_HYBRID, reg_par=[0.1, 1]')
plt.xlabel('x')
plt.ylabel('y')
plt.subplot(133)
plt.imshow(rec6[0], cmap='gray', interpolation='none')
plt.title('OSPML_HYBRID, reg_par=[0.1, 0.1]')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

#### Initialization of iterative algorithms

In [ ]:
rec0 = tomopy.recon(prj, ang, algorithm='gridrec', 
                    num_gridx=128, num_gridy=128)

rec7 = tomopy.recon(prj, ang, algorithm='ospml_hybrid', 
                    num_gridx=128, num_gridy=128,
                    num_iter=1,
                    reg_par=[0.1, 0.1],
                    num_block=50)

rec8 = tomopy.recon(prj, ang, algorithm='ospml_hybrid', 
                    num_gridx=128, num_gridy=128,
                    num_iter=1,
                    reg_par=[0.1, 0.1],
                    num_block=50,
                    init_recon=np.abs(rec0.copy()))

In [ ]:
plt.figure(dpi=150, tight_layout=True)
plt.subplot(131)
plt.imshow(rec0[0], cmap='gray', interpolation='none')
plt.title('FBP')
plt.xlabel('x')
plt.ylabel('y')
plt.subplot(132)
plt.imshow(rec7[0], cmap='gray', interpolation='none')
plt.title('Initiated with constant vals.')
plt.xlabel('x')
plt.subplot(133)
plt.ylabel('y')
plt.imshow(rec8[0], cmap='gray', interpolation='none')
plt.title('Initiated with FBP')
plt.xlabel('x')
plt.ylabel('y')
plt.show()